# Task 3: Event Impact Modeling

Ethiopia Financial Inclusion Forecasting — Event-Indicator association and impact modeling.

## Setup & Data Load

In [ ]:
import sys, os
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-whitegrid')

DATA_PATH = Path('../data/processed/ethiopia_fi_unified_data_combined.csv')
df = pd.read_csv(DATA_PATH)

# Normalize date columns
for c in ['observation_date','event_date','date','record_date']:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors='coerce')

print('Loaded rows:', len(df))
print('Columns:', sorted(df.columns))


## Separate Records by Type

In [ ]:
records = {
    'observations': df[df['record_type']=='observation'].copy(),
    'events': df[df['record_type']=='event'].copy(),
    'impact_links': df[df['record_type']=='impact_link'].copy(),
    'targets': df[df['record_type']=='target'].copy() if 'target' in df['record_type'].unique() else pd.DataFrame()
}
for k,v in records.items():
    print(k, len(v))

# Ensure key columns
imp = records['impact_links']
if 'related_indicator' not in imp.columns and 'indicator_code' in imp.columns:
    imp['related_indicator'] = imp['indicator_code']


## Event-Indicator Association Matrix

In [ ]:
events = records['events']
imp = records['impact_links']

# Select effect column: impact_magnitude preferred, fallback to impact_estimate or direction
effect_col = 'impact_magnitude' if 'impact_magnitude' in imp.columns else ('impact_estimate' if 'impact_estimate' in imp.columns else None)
if effect_col is None:
    imp['effect_value'] = imp['impact_direction'].map({'positive':1,'negative':-1}).fillna(0)
else:
    imp['effect_value'] = pd.to_numeric(imp[effect_col], errors='coerce').fillna(0)

# Join event details
key_cols = [c for c in ['record_id','event_name','category','event_date'] if c in events.columns]
ev = events[key_cols].rename(columns={'record_id':'event_id'})
link = imp.rename(columns={'parent_id':'event_id'})
merged = pd.merge(link, ev, on='event_id', how='left')

# Pivot: rows=event_name(or id), cols=related_indicator, values=mean effect
row_label = 'event_name' if 'event_name' in merged.columns else 'event_id'
col_label = 'related_indicator'
assoc = merged.pivot_table(index=row_label, columns=col_label, values='effect_value', aggfunc='mean', fill_value=0)

plt.figure(figsize=(12,6))
sns.heatmap(assoc, cmap='RdBu_r', center=0, annot=False)
plt.title('Event-Indicator Association (mean effect)')
plt.tight_layout()

# Save figure
fig_path = Path('../reports/figures/event_indicator_heatmap.png')
os.makedirs(fig_path.parent, exist_ok=True)
plt.savefig(fig_path)
print('Saved heatmap to', fig_path)

assoc.head()


## Modeling Event Impacts Over Time

In [ ]:
from dataclasses import dataclass
from typing import Optional, Dict

@dataclass
class EventEffect:
    event_id: int
    event_date: pd.Timestamp
    indicator: str
    effect_value: float
    lag_months: int = 0
    decay_months: Optional[int] = None  # if provided, effect decays linearly to 0


def build_event_effects(imp: pd.DataFrame, events: pd.DataFrame) -> pd.DataFrame:
    link = imp.rename(columns={'parent_id':'event_id'})
    key_cols = [c for c in ['record_id','event_date'] if c in events.columns]
    ev = events[key_cols].rename(columns={'record_id':'event_id'})
    merged = pd.merge(link, ev, on='event_id', how='left')
    # Resolve effect value
    effect_col = 'impact_magnitude' if 'impact_magnitude' in merged.columns else ('impact_estimate' if 'impact_estimate' in merged.columns else None)
    if effect_col is None:
        merged['effect_value'] = merged['impact_direction'].map({'positive':1,'negative':-1}).fillna(0)
    else:
        merged['effect_value'] = pd.to_numeric(merged[effect_col], errors='coerce').fillna(0)
    # lag
    if 'lag_months' in merged.columns:
        merged['lag_months'] = pd.to_numeric(merged['lag_months'], errors='coerce').fillna(0).astype(int)
    else:
        merged['lag_months'] = 0
    # indicator
    if 'related_indicator' not in merged.columns and 'indicator_code' in merged.columns:
        merged['related_indicator'] = merged['indicator_code']
    return merged[['event_id','event_date','related_indicator','effect_value','lag_months']]


def apply_event_effects_series(obs: pd.DataFrame, effects: pd.DataFrame, indicator_code: str) -> pd.DataFrame:
    # Build monthly index from min to max
    date_col = 'observation_date' if 'observation_date' in obs.columns else ('date' if 'date' in obs.columns else None)
    o = obs[(obs['indicator_code']==indicator_code) & obs[date_col].notna()].copy()
    if o.empty:
        return pd.DataFrame()
    o[date_col] = pd.to_datetime(o[date_col])
    idx = pd.date_range(o[date_col].min().floor('D'), o[date_col].max().ceil('D'), freq='MS')
    base = o.set_index(date_col)['value_numeric'].resample('MS').ffill()
    pred = base.copy().astype(float)

    # Apply step effects at event_date + lag
    effs = effects[effects['related_indicator']==indicator_code]
    for _, row in effs.iterrows():
        start = (row['event_date'] + pd.offsets.MonthBegin(1) + pd.DateOffset(months=int(row['lag_months']))).to_period('M').to_timestamp()
        if start in pred.index:
            pred.loc[start:] = pred.loc[start:] + row['effect_value']
    return pd.DataFrame({'base': base, 'predicted': pred})


## Validation: Telebirr and M-Pesa effects on Mobile Money Accounts

In [ ]:
events = records['events']
imp = records['impact_links']
obs = records['observations']

# Build effects
fx = build_event_effects(imp, events)

# Choose indicator for mobile money account ownership
indicator_mm = 'ACC_MM_ACCOUNT' if (obs['indicator_code']=='ACC_MM_ACCOUNT').any() else imp.get('related_indicator').dropna().unique()[0]
series = apply_event_effects_series(obs, fx, indicator_mm)
if not series.empty:
    fig, ax = plt.subplots(figsize=(10,4))
    series['base'].plot(ax=ax, label='Observed (ffill)', color='gray')
    series['predicted'].plot(ax=ax, label='Event-adjusted prediction', color='blue')
    ax.set_title(f'Mobile Money Account Ownership — Event-adjusted prediction ({indicator_mm})')
    ax.legend()
    plt.tight_layout()
    outp = Path('../reports/figures/mm_event_prediction.png')
    plt.savefig(outp)
    print('Saved validation figure to', outp)
else:
    print('No observation series available for indicator:', indicator_mm)


## Methodology, Assumptions, and Sources

- Effects modeled as step changes at event_date + lag_months.
- When `impact_magnitude` is unavailable, direction (+/-) is used with unit effect.
- Percentage indicators treated additively in pp; counts additively in units.
- Validation compares event-adjusted predictions to observed trajectories (e.g., 2021→2024 ACC_MM_ACCOUNT).
- Limitations: sparse observation frequency, potential mismatch between supply-side events and demand-side responses, effect heterogeneity not captured.

## Patch: Robust Effect Selection and Event Date Handling

In [ ]:
import pandas as pd

def _select_effect_column(df: pd.DataFrame):
    for c in ['impact_magnitude','impact_estimate']:
        if c in df.columns:
            s = pd.to_numeric(df[c], errors='coerce')
            if s.notna().any() and (s.abs() > 0).any():
                return c
    return None

# Recompute association matrix with robust selection
imp = records['impact_links'].copy()
col = _select_effect_column(imp)
if col is None:
    imp['effect_value'] = imp['impact_direction'].map({'positive':1,'negative':-1}).fillna(0)
    print('Effect column fallback: impact_direction → unit effect')
else:
    imp['effect_value'] = pd.to_numeric(imp[col], errors='coerce').fillna(0)
    print('Using effect column:', col)

link = imp.rename(columns={'parent_id':'event_id'})
ev = records['events'].rename(columns={'record_id':'event_id'})
# Determine event date column
edc = next((c for c in ['event_date','observation_date','date','period_start'] if c in ev.columns), None)
if edc is None:
    ev['event_date'] = pd.NaT
    print('No event date column found; event_date set to NaT')
else:
    ev['event_date'] = pd.to_datetime(ev[edc], errors='coerce')
    print('Using event date column:', edc)

merged = pd.merge(link, ev[['event_id','event_name','event_date','category']] if 'event_name' in ev.columns else ev[['event_id','event_date','category']], on='event_id', how='left')
if 'related_indicator' not in merged.columns and 'indicator_code' in merged.columns:
    merged['related_indicator'] = merged['indicator_code']
row_label = 'event_name' if 'event_name' in merged.columns else 'event_id'
assoc = merged.pivot_table(index=row_label, columns='related_indicator', values='effect_value', aggfunc='mean', fill_value=0)
assoc.to_csv(Path('../reports/association_matrix.csv'))
print('Saved association matrix to ../reports/association_matrix.csv')


## Patch: Robust build_event_effects()

In [ ]:
def build_event_effects(imp: pd.DataFrame, events: pd.DataFrame) -> pd.DataFrame:
    link = imp.rename(columns={'parent_id':'event_id'}).copy()
    ev = events.rename(columns={'record_id':'event_id'}).copy()
    edc = next((c for c in ['event_date','observation_date','date','period_start'] if c in ev.columns), None)
    if edc is None:
        ev['event_date'] = pd.NaT
    else:
        ev['event_date'] = pd.to_datetime(ev[edc], errors='coerce')
    merged = pd.merge(link, ev[['event_id','event_date']], on='event_id', how='left')
    col = _select_effect_column(merged)
    if col is None:
        merged['effect_value'] = merged['impact_direction'].map({'positive':1,'negative':-1}).fillna(0)
    else:
        merged['effect_value'] = pd.to_numeric(merged[col], errors='coerce').fillna(0)
    merged['lag_months'] = pd.to_numeric(merged['lag_months'], errors='coerce').fillna(0).astype(int) if 'lag_months' in merged.columns else 0
    if 'related_indicator' not in merged.columns and 'indicator_code' in merged.columns:
        merged['related_indicator'] = merged['indicator_code']
    # Ensure columns exist
    for req in ['event_id','event_date','related_indicator','effect_value','lag_months']:
        if req not in merged.columns:
            merged[req] = pd.NA
    return merged[['event_id','event_date','related_indicator','effect_value','lag_months']]


## Re-run Validation with Robust Effects

In [ ]:
fx = build_event_effects(records['impact_links'], records['events'])
indicator_mm = 'ACC_MM_ACCOUNT' if (records['observations']['indicator_code']=='ACC_MM_ACCOUNT').any() else fx['related_indicator'].dropna().unique()[0]
series = apply_event_effects_series(records['observations'], fx, indicator_mm)
if not series.empty:
    fig, ax = plt.subplots(figsize=(10,4))
    series['base'].plot(ax=ax, label='Observed (ffill)', color='gray')
    series['predicted'].plot(ax=ax, label='Event-adjusted prediction', color='blue')
    ax.set_title(f'Mobile Money Account Ownership — Event-adjusted prediction ({indicator_mm})')
    ax.legend()
    plt.tight_layout()
    outp = Path('../reports/figures/mm_event_prediction.png')
    plt.savefig(outp)
    print('Saved validation figure to', outp)
else:
    print('No observation series available or event dates missing; cannot validate.')
